In [3]:
import pandas as pd
import requests
import os
from pprint import pprint
import json
import numpy as np

In [4]:
n = 167
page = 0
response = requests.get(f'https://marketplace.ifood.com.br/v2/merchants?latitude=-16.6912533&longitude=-49.0845761&page={page}&channel=IFOOD&size={n}&features=&categories=&payment_types=&delivery_fee_from=0&delivery_fee_to=25&delivery_time_from=0&delivery_time_to=240')

In [5]:
response.text

:"senador-canedo-go/kero-horta-delivery-conjunto-uirapuru","userRating":0.0,"priceRange":"CHEAPEST","mainCategory":{"code":"LCH","name":"Lanches"},"features":["DELIVERY","IMMEDIATE_ORDER"],"tags":["ADDRESS_PREFORM_TYPE","CPGN_USER_DISCOUNT_BR_LIST","GUIDED_HELP_TYPE","HAMBURGUER_TAXA_GRATIS","JETSKI_BURGER_20AGO","jetski_burger_ago20","NOVO","PRATOS_SOPAS_CALDOS","RANKING_FASTFOODS_FEV21"],"paymentCodes":["DNR","MPAY","MOVPAY_MC","MC","GPY_ELO","ELOD","GPY_MCMA","AM","MCMA","MOVPAY_AM","APL_ELOD","IMV","MOVPAY_VIS","GPY_ELOD","APL_MCMA","GPY_MC","APL_ELO","MOVPAY_HIPER","APL_VISE","HIPER","IFE","VIS","APL_VIS","MOVPAY_ELO","GPY_VISE","MOVPAY_DNR","VISE","ELO","APL_MC","GPY_VIS"],"minimumOrderValue":35.0,"deliveryFee":{"type":"FIXED","value":0.0,"originalValue":0.0},"deliveryTime":59,"distance":2.02,"available":false,"takeoutTime":0,"preparationTime":0,"deliveryMethods":[{"id":"DEFAULT","priority":1,"type":"FIXED","value":0.0,"originalValue":0.0,"title":"Padrão","subtitle":"O entregador

In [6]:
array = []
n = 167
for i in range(0,2):

    r = requests.get(f'https://marketplace.ifood.com.br/v2/merchants?latitude=-16.6912533&longitude=-49.0845761&page={page}&channel=IFOOD&size={n}&features=&categories=&payment_types=&delivery_fee_from=0&delivery_fee_to=25&delivery_time_from=0&delivery_time_to=240')
    js = json.loads(r.text)
    array += js['merchants']


In [7]:
array
len(array)
lojas = array

In [8]:
float(lojas[0]['minimumOrderValue'])

15.0

In [9]:
def get_data(json) -> list:
    '''
    Função para pegar os dados da lista
    '''
    try:
        id_ = json['id'].strip()
    except:
        id_ = ''

    try:
        nome = json['name'].strip()
    except:
        nome = ''
    try:
        resource = json['resources'][0]['fileName'].strip()
    except:
        resource = ''
    try:
        localizacao = json['slug'].split('/')[0].strip()
    except:
        localizacao = ''       
    try:
        categoria = json['mainCategory']['name'].strip()
    except:
        categoria = ''
    try:
        tx_delivery = json['deliveryFee']['value']
    except:
        tx_delivery = ''
    try:
        pedido_min = float(json['minimumOrderValue'])
    except:
        pedido_min = ''      

    variaveis = locals()
    variaveis.pop('json')

    return variaveis.values()

In [10]:
get_data(lojas[0])

dict_values(['95c7c20a-728e-4984-b39c-035f6e291d2a', 'Gold Burger Sanduicheria', '95c7c20a-728e-4984-b39c-035f6e291d2a/201907261921_loke_.jpeg', 'senador-canedo-go', 'Lanches', 0.0, 15.0])

In [13]:
res = list()
for json in lojas:
    res.append(get_data(json))
df = pd.DataFrame(res, columns=['id', 'nome', 'resource', 'localizacao', 'categoria', 'tx_delivery', 'pedido_minimo'])
df.head()    

,id,nome,resource,localizacao,categoria,tx_delivery,pedido_minimo
0,95c7c20a-728e-4984-b39c-035f6e291d2a,Gold Burger Sanduicheria,95c7c20a-728e-4984-b39c-035f6e291d2a/201907261...,senador-canedo-go,Lanches,0.0,15
1,b06ce7bd-6386-43d9-a785-4b3d9b577d8c,Nativos Açaí,b06ce7bd-6386-43d9-a785-4b3d9b577d8c/202007251...,senador-canedo-go,Açaí,0.0,8
2,3747779d-967e-4b49-8278-646e63a39dbf,Casa do Açaí,3747779d-967e-4b49-8278-646e63a39dbf/202103092...,senador-canedo-go,Açaí,0.0,10
3,3591705c-1f61-4e17-8844-05dcc90362e0,Espetos Grill 🍖,3591705c-1f61-4e17-8844-05dcc90362e0/202012301...,senador-canedo-go,Brasileira,3.0,10
4,cc4bdc76-656a-4670-8802-b023fae92378,D Esfihas,cc4bdc76-656a-4670-8802-b023fae92378/202009202...,senador-canedo-go,Lanches,5.0,10


In [14]:
url = 'https://static-images.ifood.com.br/image/upload/t_thumbnail/logosgde/'

In [15]:
df['resource'] = df['resource'].apply(lambda x: url+x)

In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 334 entries, 0 to 333
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             334 non-null    object 
 1   nome           334 non-null    object 
 2   resource       334 non-null    object 
 3   localizacao    334 non-null    object 
 4   categoria      334 non-null    object 
 5   tx_delivery    334 non-null    float64
 6   pedido_minimo  334 non-null    object 
dtypes: float64(1), object(6)
memory usage: 18.4+ KB


In [17]:
df['pedido_minimo'] = pd.to_numeric(df['pedido_minimo'], downcast='float')

In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 334 entries, 0 to 333
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             334 non-null    object 
 1   nome           334 non-null    object 
 2   resource       334 non-null    object 
 3   localizacao    334 non-null    object 
 4   categoria      334 non-null    object 
 5   tx_delivery    334 non-null    float64
 6   pedido_minimo  332 non-null    float32
dtypes: float32(1), float64(1), object(5)
memory usage: 17.1+ KB


In [19]:
def download_logos(id_: str, resource: str) -> None:
    '''
    Função para fazer download localmente das logos
    '''
    pics_path = 'pics'

    if not os.path.exists(pics_path):
        os.mkdir(pics_path)

    posfix = resource.split('.')[-1]
    file_name = f'{id_}.{posfix}'

    file_path = os.path.join(os.getcwd(), pics_path, file_name)

    try:
        response = requests.get(resource)
    except:
        pass
    else:
        with open(file_path, 'wb') as f:
            f.write(response.content)

In [20]:
os.getcwd()

'd:\\Projetos\\Pessoais\\py\\python_intermediario'

In [21]:
download_logos('teste', df.iloc[0].resource)

In [22]:
for _, row in df.iterrows():
    download_logos(row.id, row.resource)

In [23]:
df.shape

(334, 7)

In [24]:
df.to_csv('dados_lj.csv', index=False)